In [2]:
# 数据清洗学习目标：
# 1、知道什么事缺失值、为什么会产生缺失值
# 2、掌握缺失值、重复数据、异常数据的处理方式

In [5]:
# 空数据验证
from numpy import NaN, NAN, nan
print(NaN==True)
print(NaN==False)
print(NaN==0)
print(NaN=='')

print(NaN==NaN)
print(NaN==nan)
print(NaN==NAN)
print(nan==NAN)


In [6]:
# 导入Pandas包
import pandas as pd

In [9]:
# Pandas提供了isnull/isna方法，用于测试某个值是否为缺失值
print(pd.isnull(NaN))
print(pd.isnull(nan))
print(pd.isnull(NAN))

print(pd.isna(NaN))
print(pd.isna(nan))
print(pd.isna(NAN))

In [10]:
# Pandas的notnull/notna方法也可以用于判断某个值是否为缺失值
print(pd.notnull(NaN))
print(pd.notnull(42))


In [17]:
# 加载包含缺失的数据

# 默认数据加载
survey_visited = pd.read_csv('../data/survey_visited.csv')
print(survey_visited.head(10))

# 加载数据不包含默认缺失值
survey_visited_default = pd.read_csv('../data/survey_visited.csv', keep_default_na = False)
print(survey_visited_default.head(10))

# 记载数据手动指定默认值
survey_visited_default_handle = pd.read_csv('../data/survey_visited.csv', na_values = ['空值'], keep_default_na = False)
# print(survey_visited_default_handle.head(10))
survey_visited_default_handle

In [21]:
# 处理缺失值

# 1、创建数据
data = [
    (1, 143.5, 5.6, 28,'M', 100000),
    (2, 167.2, 5.4, 45,'M', None),
    (3, None , 5.2, None, None, None),
    (4, 144.5, 5.9, 33, 'M', None),
    (5, 133.2, 5.7, 54, 'F', None),
    (6, 124.1, 5.2, None, 'F', None),
    (7, 129.2, 5.3, 42, 'M', 76000),
]

columns = ['id', 'weight', 'height', 'age', 'gender', 'income']


df = pd.DataFrame(data = data, columns = columns)

In [24]:
# 直接使用dropna删掉包含缺失值的数据
print(df.dropna())

# 可以设定参数subset和how
# subset：指定那些列包含空值
# how：有两个参数，any：包含任意个缺失值就删除一行；all：全列都是缺失值才删掉一行

subset = ['age', 'income']
print(df.dropna(subset=subset, how='all'))

In [32]:
# 非时间序列数据缺失值处理

# 使用常量来替换默认值
# print(df.fillna(0))

# 使用统计量来替换（缺失值所处的列平均值、中位数、众数）
df['age'].fillna(df['age'].mean())

In [33]:
# 时间序列数据缺失值处理
city_day = pd.read_csv('../data/city_day.csv', index_col = 'Date')
city_day

In [41]:
# 数据切片
city_day['Xylene'][54:]

In [42]:
# 查看包含缺失数据的部分
print(city_day['Xylene'][54:61])

# 使用fill填充，用时间序列中空值上一个非空值进行填充
city_day.fillna(method = 'ffill')['Xylene'][54:61]

In [43]:
# 使用bfill填充
city_day.fillna(method='bfill')['Xylene'][54:61]


In [43]:
# 线性插值方法填充缺失值
# 1、时间序列数据，数据随着时间的变化可能会较大。 因此，使用bfill和ffill进行插补并不是解决缺失值问题的最优方案。
# 2、线性插值法是一种插补缺失值技术，它假定数据点之间存在线性关系，并利用相邻数据点中的非缺失值来计算缺失数据点的值。

In [44]:
city_day.interpolate(limit_direction = 'both')['Xylene'][54:61]

In [45]:
# 重复数据处理

# 1、构造数据
data = [
    (1, 144.5, 5.9, 33, 'M'),
    (2, 167.2, 5.4, 45, 'M'),
    (3, 124.1, 5.2, 23, 'F'),
    (4, 144.5, 5.9, 33, 'M'),
    (5, 133.2, 5.7, 54, 'F'),
    (3, 124.1, 5.2, 23, 'F'),
    (5, 129.2, 5.3, 42, 'M'),
]

columns=['id', 'weight', 'height', 'age', 'gender']

df2 = pd.DataFrame(data = data, columns =columns)
df2

In [48]:
# 删除重复数据
df2.drop_duplicates()

In [49]:
# 删除指定列中完全一样的行
subset=['weight', 'height', 'age', 'gender']
df2.drop_duplicates(subset = subset)

In [75]:
import hashlib

# 定义一个函数，用于生成哈希值
def generate_hash(text):
    return hashlib.md5(text.encode()).hexdigest()

# 通过hash生成新的ID
df2['newId'] = (df2['gender']).apply(generate_hash)
df2

In [76]:
# 异常值处理

# 1、构造数据
data=[
    (1, 143.5, 5.3, 28),
    (2, 154.2, 5.5, 45),
    (3, 342.3, 5.1, 99),
    (4, 144.5, 5.5, 33),
    (5, 133.2, 5.4, 54),
    (6, 124.1, 5.1, 21),
    (7, 129.2, 5.3, 42),
]

columns = ['id', 'weight', 'height', 'age']

df3 = pd.DataFrame(data = data, columns = columns)
df3


In [99]:
# 计算边界值

# 1、定义字段名称
columns = ['weight', 'height', 'age']

# 2、定义回归系数（quantile() 函数是获取行或列的分位数）
quantile = df3[columns].quantile(axis = 0, q = [0.25, 0.75])
print("体重、身高、年龄的1/4 3/4 分位数：", quantile)

bounds = {}
for col in columns:
    IQR = quantile[col][0.75] - quantile[col][0.25]
    print(col, IQR)
    bounds[col] = [quantile[col][0.25]-1.5*IQR, quantile[col][0.75]+1.5*IQR]


In [100]:
# 边界值
bounds

In [91]:
# 1、异常值判断条件
abnormal = [(df[c] > bounds[c][1]) | (df[c] < bounds[c][0])  for c in columns]

# 2、合并异常值
df_outer = pd.concat(abnormal, axis = 1)

# 3、删除非异常值
df3[df_outer].dropna(how='all')